In [170]:
with open('15.txt') as f:
    grid, instrs = f.read().split('\n\n')
    instrs = instrs.replace('\n', '').strip()
    grid = grid.strip()

DIRECTIONS = {
    'v': ( 0, +1),
    '^': ( 0, -1),
    '<': (-1,  0),
    '>': (+1,  0),
}

In [171]:
# We'll account for `[` so we don't need to repeat ourselves:

Coord = tuple[int, int]
def to_dict(grid: str):
    robot = None
    obstacles: dict[Coord, str] = dict()
    for y, line in enumerate(grid.splitlines()):
        for x, char in enumerate(line):
            if char == '@':
                robot = x, y
            elif char in 'O#[':
                obstacles[x, y] = char
    assert robot
    return robot, obstacles

def to_str(robot: Coord, obstacles: dict[Coord, str]):
    xs, ys = list(zip(*obstacles.keys()))
    grid = [['.' for _ in range(max(xs)+1)] for _ in range(max(ys)+1)]
    for (x, y), char in obstacles.items():
        grid[y][x] = char
    grid[robot[1]][robot[0]] = '@'
    
    return '\n'.join(''.join(line) for line in grid).replace('[.', '[]')

assert grid == to_str(*to_dict(grid))

# Part 1: Single-width

In [172]:
def push(robot: Coord, obstacles: dict[Coord, str], dir: Coord, box_width=1, box_char='O'):
    rx, ry = robot
    dx, dy = dir
    moving_boxes = []
    bx, by = rx+dx, ry+dy

    if obstacles.get((bx-1, by)) == '[':
        bx -= 1

    while (char := obstacles.get((bx, by), '.')) not in '#.':
        moving_boxes.append((bx, by))
        bx += dx * box_width
        by += dy

    if char == '#' and box_width == 2:
        # print('EVIL BEHAVIOUR', obstacles.get((bx-dx, by), '.'), dx)
        # due to the way we've coded this we're in a very evil and subtle little behaviour.
        # we've pushed to the edge but is there a single gap to push into?
        assert dy == 0
        if moving_boxes and obstacles.get((bx-dx, by), '.') == '.':
            char = '.'
            if dx == 1:
                # print('DOUBLE EVIL BEHAVIOUR')
                # print(+1, obstacles.get((bx+1, by), '.'))
                # print(0, obstacles.get((bx, by), '.'))
                # print(-1, obstacles.get((bx-1, by), '.'))
                # print(-2, obstacles.get((bx-2, by), '.'))
                if obstacles.get((bx-2, by), '.') == '[':
                    char = ']'
        # print('evil behaviour.')
    
    if char == '.':
        if moving_boxes:
            pass
            # print('left' if dx == -1 else 'right', moving_boxes)
            # print(to_str((rx, ry), obstacles))
        rx += dx
        ry += dy
        for bx, by in moving_boxes:
            obstacles.pop((bx, by))
        for bx, by in moving_boxes:
            obstacles[(bx+dx, by+dy)] = box_char
    
    return (rx, ry), obstacles

def move1(robot: Coord, obstacles: dict[Coord, str], instrs: list[str]):
    for d in instrs:
        robot, obstacles = push(robot, obstacles, DIRECTIONS.get(d))
    
    return robot, obstacles

In [173]:
robot, obstacles = move1(*to_dict(grid), instrs)
sum(y*100 + x for (x, y), char in obstacles.items() if char == 'O')


1398947

# Part 2: Double Width

In [174]:
grid2 = grid.replace('#', '##').replace('O', '[]').replace('.', '..').replace('@', '@.')

In [175]:
def get_obstacles(obstacles: dict[Coord, str], xy: Coord, dy: int):
    x, y = xy

    char = obstacles.get((x, y+dy), None)
    if not char:
        x -= 1
        char = obstacles.get((x, y+dy), None)
        if char != '[':
            return

    yield (x, y+dy, char)
    if char == '#':
        return
    if char == '[':
        yield from get_obstacles(obstacles, (x, y+dy), dy)
        yield from get_obstacles(obstacles, (x+1, y+dy), dy)

def move2(robot: Coord, obstacles: dict[Coord, str], instrs: list[tuple[int, int]]):
    rx, ry = robot
    for direction in instrs:
        dx, dy = DIRECTIONS.get(direction)
        if dy == 0:
            (rx, ry), obstacles = push((rx, ry), obstacles, (dx, dy), box_width=2, box_char='[')
        else:
            things_to_push = set(get_obstacles(obstacles, (rx, ry), dy))
            if any(c == '#' for _,_,c in things_to_push):
                show((rx, ry), obstacles, direction)
                continue
            moving_boxes = [(bx, by) for bx, by, _ in things_to_push]
            for bx, by in moving_boxes:
                obstacles.pop((bx, by))
            for bx, by in moving_boxes:
                obstacles[(bx, by+dy)] = '['
            rx += dx
            ry += dy
        show((rx, ry), obstacles, direction)
    
    return (rx, ry), obstacles


robot, obstacles = move2(*to_dict(grid2), instrs)
# print(to_str(robot, obstacles))

sum(y*100 + x for (x, y), char in obstacles.items() if char == '[')

1397393